# Analysis outline

The analysis has the following components:

1. [orthologs.ipynb](orthologs.ipynb) collect primate orthologs of each human gene from NCBI, remove redundant sequences, save them to a `*.fasta` file.  

1. [align.ipynb](align.ipynb) align orthologous sequences in each `*.fasta` file and save it to a `*.aln` file.

1. [pathogen.ipynb](pathogen.ipynb) extract pathogenic variants from ClinVar data. 

1. [clean_up.ipynb](clean_up.ipynb) establish connections between transcripts in ClinVar data and proteins in ortholog collections.

1. [find_CPDs.ipynb](find_CPDs.ipynb) find putative CPDs.

# Set up

Run the following cell.  It loads some required functions.

In [2]:
run kondrashov

Run the following two cell.  The first displays the path to the current folder.  The second displays the contents of that folder.  (Note: your output will be different from mine.)

In [34]:
pwd

'/Users/arnav/hhh Dropbox/arnav mehta/Mac/Documents/GitHub/kondrashov'

In [35]:
ls

ACE2 ortholog.ipynb    clean_up.ipynb         orthologs-Copy1.ipynb
README.md              fasta/                 orthologs.ipynb
align.ipynb            find_CPDs.ipynb        pathogen.ipynb
aln/                   kondrashov.py


If there is no folder called `fasta`, create it by running the following command.

In [36]:
mkdir fasta

mkdir: fasta: File exists


If there is no folder called `aln`, create it by running the following command.

In [37]:
mkdir aln

mkdir: aln: File exists


Rerun the `ls` command above to confirm that you have created those folders successfully. 

The following defines a function that gets the species name from a NCBI sequence record.

# Get protein sequences for primate orthologs of human genes

**Note: this is a new version of the `orthologs.ipynb` notebook that prevents unique human sequences from being dropped.**

* Enter the gene name you are working on in the following cell.  In this page I'll use ALPL as an example.

## ACE2

In [4]:
# enter gene name here
gene = 'ACE2'

* In the [NCBI query page](https://www.ncbi.nlm.nih.gov/labs/gquery/) search for `Homo sapiens ALPL`.  

* When the search results come up ([this is the page for ALPL](https://www.ncbi.nlm.nih.gov/labs/gquery/all/?term=Homo+sapiens+ALPL)) click on `Orthologs`.  Here's what the output looks like for [ALPL](https://www.ncbi.nlm.nih.gov/gene/249/ortholog/?scope=9443&term=ALPL).

* Use the Taxonomy Tree to select **mammals > placentals > primates**.  For ALPL, this reduces the number of orthologs from 335 to 29.

* In the list of sequences click to select all (box labelled `0 selected`).

* Click on `Add to cart`.  The items appear in a box with a shopping cart symbol.  Click on that box.

* A popup window appears.  Click on `Protein alignment`.  Choose `all sequences per gene`.  Click `Align`.

* Select all the accession numbers in the box (XP_039330133.1, XP_037856105.1, etc) and copy them.

* Run the following cell (it pulls the names from the clipboard and puts them on a list).

In [5]:
# copy IDs from NCBI ortholog protein alignment dialog (all sequences per gene)
tmp = pd.read_clipboard(sep= '/n', names=['id'])
seq_ids = tmp['id'].tolist()
print('{0} orthologs: {1} primate sequences (retrieved on {2})\n'.format(gene, len(seq_ids), date.today()))
print(seq_ids)

ACE2 orthologs: 28 primate sequences (retrieved on 2022-07-21)

['NP_001358344.1', 'NP_001129168.1', 'XP_016798468.1', 'XP_007989304.2', 'XP_008987241.1', 'XP_011891198.1', 'XP_005593094.1', 'XP_011733505.1', 'XP_021788732.1', 'XP_025227847.1', 'XP_011850923.1', 'XP_033056809.1', 'XP_017744069.1', 'XP_010364367.2', 'XP_023054821.1', 'XP_018874749.1', 'XP_008972428.2', 'NP_001124604.1', 'XP_003261132.2', 'XP_032612508.1', 'XP_010334925.2', 'XP_032141854.1', 'XP_017367865.1', 'XP_012290105.1', 'XP_008062810.1', 'XP_012494185.1', 'XP_020140826.1', 'XP_003791912.1']


In [6]:
# you can do it by hand if you already have all the sequences
seq_ids = ['NP_001358344.1', 'NP_001129168.1', 'XP_016798468.1', 'XP_007989304.2', 'XP_008987241.1', 'XP_011891198.1', 'XP_005593094.1', 'XP_011733505.1', 'XP_021788732.1', 'XP_025227847.1', 'XP_011850923.1', 'XP_033056809.1', 'XP_017744069.1', 'XP_010364367.2', 'XP_023054821.1', 'XP_018874749.1', 'XP_008972428.2', 'NP_001124604.1', 'XP_003261132.2', 'XP_032612508.1', 'XP_010334925.2', 'XP_032141854.1', 'XP_017367865.1', 'XP_012290105.1', 'XP_008062810.1', 'XP_012494185.1', 'XP_020140826.1', 'XP_003791912.1']
print('{0} orthologs: {1} primate sequences\n'.format(gene, len(seq_ids)))
print(seq_ids)

ACE2 orthologs: 28 primate sequences

['NP_001358344.1', 'NP_001129168.1', 'XP_016798468.1', 'XP_007989304.2', 'XP_008987241.1', 'XP_011891198.1', 'XP_005593094.1', 'XP_011733505.1', 'XP_021788732.1', 'XP_025227847.1', 'XP_011850923.1', 'XP_033056809.1', 'XP_017744069.1', 'XP_010364367.2', 'XP_023054821.1', 'XP_018874749.1', 'XP_008972428.2', 'NP_001124604.1', 'XP_003261132.2', 'XP_032612508.1', 'XP_010334925.2', 'XP_032141854.1', 'XP_017367865.1', 'XP_012290105.1', 'XP_008062810.1', 'XP_012494185.1', 'XP_020140826.1', 'XP_003791912.1']


* Run the next cell.  It pulls the sequences from NCBI and grabs any unique human sequences.  Then it flags any redundant sequences and drops those.  For ALPL it drops 84 sequences and we end up with 57.

In [7]:
# 1. Collect unique human sequences
print('{0} orthologs: all primate sequences\n'.format(gene))
all_sequences = []
seq_records = []
inc = 0
exc = 0
with Entrez.efetch(
    db="protein", rettype="gb", retmode="text", id=seq_ids,
) as handle:
    for seq_record in SeqIO.parse(handle, "gb"):
        sp = get_species(seq_record)
        if sp == 'Homo sapiens':
            seq = seq_record.seq
            if seq in all_sequences:
                print("\t{0}\t({1} aa)\t{2}\t*** the same as sequence {3} (excluded) ***".format(seq_record.id, len(seq), seq_record.description, all_sequences.index(seq)))
                exc += 1
            else:
                all_sequences.append(seq)
                print("{3}:\t{0}\t({1} aa)\t{2}".format(seq_record.id, len(seq), seq_record.description, inc))
                seq_records.append(seq_record)
                inc += 1
# # 2. Collect other unique sequences
with Entrez.efetch(
    db="protein", rettype="gb", retmode="text", id=seq_ids,
) as handle:
    for seq_record in SeqIO.parse(handle, "gb"):
        seq = seq_record.seq
        if seq in all_sequences:
            print("\t{0}\t({1} aa)\t{2}\t*** the same as sequence {3} (excluded) ***".format(seq_record.id, len(seq), seq_record.description, all_sequences.index(seq)))
            exc += 1
        else:
            all_sequences.append(seq)
            print("{3}:\t{0}\t({1} aa)\t{2}".format(seq_record.id, len(seq), seq_record.description, inc))
            seq_records.append(seq_record)
            inc += 1
print('\n\tTotal:\t', inc, ' unique sequences (', exc, ' excluded)', sep='')

ACE2 orthologs: all primate sequences

0:	NP_001358344.1	(805 aa)	angiotensin-converting enzyme 2 isoform 1 precursor [Homo sapiens]
	NP_001358344.1	(805 aa)	angiotensin-converting enzyme 2 isoform 1 precursor [Homo sapiens]	*** the same as sequence 0 (excluded) ***
1:	NP_001129168.1	(805 aa)	angiotensin-converting enzyme 2 precursor [Macaca mulatta]
2:	XP_016798468.1	(805 aa)	angiotensin-converting enzyme 2 isoform X1 [Pan troglodytes]
3:	XP_007989304.2	(805 aa)	angiotensin-converting enzyme 2 [Chlorocebus sabaeus]
4:	XP_008987241.1	(805 aa)	angiotensin-converting enzyme 2 [Callithrix jacchus]
5:	XP_011891198.1	(805 aa)	PREDICTED: angiotensin-converting enzyme 2 [Cercocebus atys]
6:	XP_005593094.1	(805 aa)	PREDICTED: angiotensin-converting enzyme 2 [Macaca fascicularis]
7:	XP_011733505.1	(805 aa)	angiotensin-converting enzyme 2 [Macaca nemestrina]
8:	XP_021788732.1	(805 aa)	angiotensin-converting enzyme 2 [Papio anubis]
9:	XP_025227847.1	(805 aa)	angiotensin-converting enzyme 2 [Thero

In [41]:
seq_records

[SeqRecord(seq=Seq('MISPFLVLAIGTCLTNSLVPEKEKDPKYWRDQAQETLKYALELQKLNTNVAKNV...VLF'), id='NP_001356734.1', name='NP_001356734', description='alkaline phosphatase, tissue-nonspecific isozyme isoform 1 preproprotein [Homo sapiens]', dbxrefs=[]),
 SeqRecord(seq=Seq('MPWSFRSSTPTWLRMSSCSWEMGQLHHNPGEETRLEMDKFPFVALSKTYNTNAQ...VLF'), id='XP_016856392.1', name='XP_016856392', description='alkaline phosphatase, tissue-nonspecific isozyme isoform X1 [Homo sapiens]', dbxrefs=['BioProject:PRJNA168']),
 SeqRecord(seq=Seq('MPWSFRSSTPTWLRMSSCSWEMTYNTNAQVPDSAGTATAYLCGVKANEGTVGVS...VLF'), id='NP_001170991.1', name='NP_001170991', description='alkaline phosphatase, tissue-nonspecific isozyme isoform 3 [Homo sapiens]', dbxrefs=[]),
 SeqRecord(seq=Seq('MFLGDGMGVSTVTAARILKGQLHHNPGEETRLEMDKFPFVALSKTYNTNAQVPD...VLF'), id='NP_001120973.2', name='NP_001120973', description='alkaline phosphatase, tissue-nonspecific isozyme isoform 2 [Homo sapiens]', dbxrefs=[]),
 SeqRecord(seq=Seq('MISPFLVLAIGTCLTNSLVPEKEKDPKYWRDQ

* Copy the output to the cell below

* The following cell writes the sequences to the file `ALPL.fasta` (fasta format).

In [8]:
# write the sequences to a fasta file
print('{0} orthologs: unique primate sequences\n'.format(gene))
i = 0
with open("fasta/{0}.fasta".format(gene), "w") as output:
    for seq_record in seq_records:
        print("{3}:\t{0}\t({1} aa)\t{2}".format(seq_record.id, len(seq), seq_record.description, i))
        SeqIO.write(seq_record, output, "fasta")
        i += 1
print("\n{0}.fasta saved!".format(gene))

ACE2 orthologs: unique primate sequences

0:	NP_001358344.1	(805 aa)	angiotensin-converting enzyme 2 isoform 1 precursor [Homo sapiens]
1:	NP_001129168.1	(805 aa)	angiotensin-converting enzyme 2 precursor [Macaca mulatta]
2:	XP_016798468.1	(805 aa)	angiotensin-converting enzyme 2 isoform X1 [Pan troglodytes]
3:	XP_007989304.2	(805 aa)	angiotensin-converting enzyme 2 [Chlorocebus sabaeus]
4:	XP_008987241.1	(805 aa)	angiotensin-converting enzyme 2 [Callithrix jacchus]
5:	XP_011891198.1	(805 aa)	PREDICTED: angiotensin-converting enzyme 2 [Cercocebus atys]
6:	XP_005593094.1	(805 aa)	PREDICTED: angiotensin-converting enzyme 2 [Macaca fascicularis]
7:	XP_011733505.1	(805 aa)	angiotensin-converting enzyme 2 [Macaca nemestrina]
8:	XP_021788732.1	(805 aa)	angiotensin-converting enzyme 2 [Papio anubis]
9:	XP_025227847.1	(805 aa)	angiotensin-converting enzyme 2 [Theropithecus gelada]
10:	XP_011850923.1	(805 aa)	PREDICTED: angiotensin-converting enzyme 2 [Mandrillus leucophaeus]
11:	XP_033056809.1

* The following cell aligns the sequences and writes them to the file `ALPL.aln` (also in fasta format).  To look at the alignment open it in [this viewer application](https://alignmentviewer.org/).  (Hannah: you may not be able to run this step.  Just skip it and repeat the sequence processing above for the other genes.)

In [9]:
# align sequences
print('{0} orthologs\n'.format(gene))
muscle_cline = MuscleCommandline(input='fasta/{0}.fasta'.format(gene))
stdout, stderr = muscle_cline()
align = AlignIO.read(StringIO(stdout), 'fasta')
AlignIO.write(align, 'aln/{0}.aln'.format(gene), 'fasta')
print(align)
print("\n{0}.aln saved!".format(gene))

ACE2 orthologs



ApplicationError: Non-zero return code 127 from 'muscle -in fasta/ACE2.fasta', message '/bin/sh: muscle: command not found'

* Repeat for other genes.

In [10]:
!open .